In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('C:/Users/udipt/Desktop/Websoft/Assignment/DataScienceCaseStudy/dataSample.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16162 entries, 0 to 16161
Data columns (total 11 columns):
Unnamed: 0               16162 non-null int64
ID                       16162 non-null int64
Type                     16162 non-null object
event_timestamp          16162 non-null int64
event_label              16162 non-null object
log_timestamp            16162 non-null int64
log_reading_1            15848 non-null float64
log_reading_2            15883 non-null object
log_reading_3            15843 non-null float64
log_reading_4            16162 non-null int64
warranty_service_flag    1389 non-null object
dtypes: float64(2), int64(5), object(4)
memory usage: 1.4+ MB


# Numeric variables

In [3]:
df=pd.get_dummies(df)
df.columns

Index(['Unnamed: 0', 'ID', 'event_timestamp', 'log_timestamp', 'log_reading_1',
       'log_reading_3', 'log_reading_4', 'Type_TypeA', 'Type_TypeB',
       'event_label_activate', 'event_label_break', 'log_reading_2_a',
       'log_reading_2_b', 'log_reading_2_c', 'warranty_service_flag_n',
       'warranty_service_flag_y'],
      dtype='object')

In [4]:
df=df.drop(columns=['warranty_service_flag_n','event_label_activate','Unnamed: 0'],axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16162 entries, 0 to 16161
Data columns (total 13 columns):
ID                         16162 non-null int64
event_timestamp            16162 non-null int64
log_timestamp              16162 non-null int64
log_reading_1              15848 non-null float64
log_reading_3              15843 non-null float64
log_reading_4              16162 non-null int64
Type_TypeA                 16162 non-null uint8
Type_TypeB                 16162 non-null uint8
event_label_break          16162 non-null uint8
log_reading_2_a            16162 non-null uint8
log_reading_2_b            16162 non-null uint8
log_reading_2_c            16162 non-null uint8
warranty_service_flag_y    16162 non-null uint8
dtypes: float64(2), int64(4), uint8(7)
memory usage: 868.2 KB


# Missing values

In [5]:
df['log_reading_1'].fillna(df['log_reading_1'].median(),inplace=True)
df['log_reading_3'].fillna(df['log_reading_3'].median(),inplace=True)

# Train-test split

In [6]:
x=df.drop('event_label_break',axis=1)
y=df['event_label_break']
print(y.mean())

0.3414800148496473


In [7]:
from sklearn.model_selection import train_test_split

In [8]:
x_tr,x_te,y_tr,y_te=train_test_split(x,y,test_size=.3,stratify=y,random_state=10)
print(x_tr.shape,x_te.shape,y_tr.shape,y_te.shape,y_tr.mean(),y_te.mean(),sep='---')

(11313, 12)---(4849, 12)---(11313,)---(4849,)---0.3414655705825157---0.3415137141678697


# XGboost

In [10]:
import xgboost as xgb

In [62]:
xg=xgb.XGBClassifier(base_score=.34,n_jobs=-1,random_state=10)
xg.fit(x_tr,y_tr)

XGBClassifier(base_score=0.34, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=-1, num_parallel_tree=1,
              objective='binary:logistic', random_state=10, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [63]:
print(f'The custom model predicts {xg.predict(x_te).mean()*100:.2f}% points as positive')

The custom model predicts 37.76% points as positive


In [12]:
from sklearn.model_selection import GridSearchCV

In [64]:
pg={'n_estimators':range(200,251,10)}
xg_gr=GridSearchCV(xg,pg,cv=5,n_jobs=-1,scoring='roc_auc')
xg_gr.fit(x_tr,y_tr)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.34, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0, gpu_id=-1,
                                     importance_type='gain',
                                     interaction_constraints='',
                                     learning_rate=0.300000012,
                                     max_delta_step=0, max_depth=6,
                                     min_child_weight=1, missing=nan,
                                     monotone_constraints='()',
                                     n_est...rs=100, n_jobs=-1,
                                     num_parallel_tree=1,
                                     objective='binary:logistic',
                                     random_state=10, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, sub

In [65]:
print(f'Best parameters are {xg_gr.best_params_}')

Best parameters are {'n_estimators': 200}


In [66]:
xg_gb=xg_gr.best_estimator_
y_pro=xg_gb.predict_proba(x_te)
y_pre=xg_gb.predict(x_te)
print(f'The model predicts {y_pre.mean()*100:.2f}% points as positive')

The model predicts 36.21% points as positive


# Result analysis

In [67]:
Test_array=[]
for i in y_pro:
    Test_array.append(i[1])
Test_array

[0.42025554,
 0.0016267279,
 0.0041177203,
 0.098133914,
 0.34862465,
 0.0018365277,
 0.458429,
 0.72252584,
 0.0059563913,
 0.99586546,
 0.01722325,
 0.0067896037,
 0.07148429,
 0.009459116,
 0.26290563,
 0.64070845,
 0.011807912,
 0.20231637,
 0.0015888875,
 0.45999336,
 0.013321938,
 0.8604137,
 0.6381177,
 0.8114269,
 0.57082725,
 0.5713725,
 0.0015670238,
 0.64506817,
 0.51694113,
 0.029473884,
 0.679631,
 0.7734546,
 0.008556421,
 0.73107016,
 0.007744525,
 0.019005701,
 0.0018353341,
 0.0048465068,
 0.986652,
 0.0019886843,
 0.001981088,
 0.6337368,
 0.24853191,
 0.6560779,
 0.6261018,
 0.68756676,
 0.33251557,
 0.0040147887,
 0.4886038,
 0.5464401,
 0.9833692,
 0.5462222,
 0.001341001,
 0.62846214,
 0.56124246,
 0.123144075,
 0.7937774,
 0.030351363,
 0.4821569,
 0.52804196,
 0.08669495,
 0.00860774,
 0.22728018,
 0.7952757,
 0.43774092,
 0.7125111,
 0.5169423,
 0.012030124,
 0.48982587,
 0.71190405,
 0.66217923,
 0.8402968,
 0.91814137,
 0.0049239756,
 0.14750254,
 0.66130996,

In [68]:
def Average(lst):
    return sum(lst) / len(lst)
average = Average(Test_array)
print(average)

0.33637233723630783


In [69]:
y_pro[1][0].mean()

0.99837327

In [70]:
y_te_pred=xg_gb.predict(x_te)
y_te_pred

array([0, 0, 0, ..., 0, 0, 1], dtype=uint8)

In [71]:
y_te_pred.mean()

0.36213652299443183

In [72]:
y_te_pred[1:11]

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0], dtype=uint8)

In [73]:
Test_array[1:11]

[0.0016267279,
 0.0041177203,
 0.098133914,
 0.34862465,
 0.0018365277,
 0.458429,
 0.72252584,
 0.0059563913,
 0.99586546,
 0.01722325]

# Performance

In [18]:
from sklearn.metrics import accuracy_score,roc_auc_score,f1_score,precision_score,recall_score

### On training data

In [19]:
print(f'Accuracy score ={accuracy_score(y_tr,xg_gb.predict(x_tr))}, roc auc score ={roc_auc_score(y_tr,xg_gb.predict(x_tr))}')

Accuracy score =0.7728112301440709, roc auc score =0.7831686435258967


In [20]:
print(f'f1 score ={f1_score(y_tr,xg_gb.predict(x_tr))}, precision ={precision_score(y_tr,xg_gb.predict(x_tr))}, recall ={recall_score(y_tr,xg_gb.predict(x_tr))}')

f1 score =0.7103838003296444, precision =0.6290658882402002, recall =0.8158464034613304


### On test data

In [21]:
print(f'Accuracy score ={accuracy_score(y_te,xg_gb.predict(x_te))}, roc auc score ={roc_auc_score(y_te,xg_gb.predict(x_te))}')

Accuracy score =0.7467191601049868, roc auc score =0.7541546086060202


In [22]:
print(f'f1 score ={f1_score(y_te,xg_gb.predict(x_te))}, precision ={precision_score(y_te,xg_gb.predict(x_te))}, recall ={recall_score(y_te,xg_gb.predict(x_te))}')

f1 score =0.6770260578532153, precision =0.5994919559695173, recall =0.7775947281713345
